# DAY 30: CUDA Dynamic Parallelism - Recursive Tree Traversal

In [ ]:
%%writefile dynamic_parallelism.cu
// nvcc -arch=sm_35 -rdc=true dynamic_parallelism.cu -o dynamic_parallelism -lcudadevrt

#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>

#define MAX_DEPTH 10
#define BLOCK_SIZE 256

// Binary tree node structure
struct TreeNode {
    int value;
    int left_idx;   // Index of left child (-1 if none)
    int right_idx;  // Index of right child (-1 if none)
};

// Device function to process a single node
__device__ void process_node(TreeNode* nodes, int node_idx, int* results, int depth) {
    if (node_idx == -1) return;
    
    // Process current node (square its value)
    atomicAdd(&results[depth], nodes[node_idx].value * nodes[node_idx].value);
}

// Child kernel for processing tree nodes
__global__ void tree_traverse_child(TreeNode* nodes, int* node_indices, int num_nodes, 
                                   int* results, int depth) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= num_nodes) return;
    
    int node_idx = node_indices[idx];
    process_node(nodes, node_idx, results, depth);
    
    // Collect children for next level
    if (depth < MAX_DEPTH - 1 && node_idx != -1) {
        int children[2];
        int child_count = 0;
        
        if (nodes[node_idx].left_idx != -1) {
            children[child_count++] = nodes[node_idx].left_idx;
        }
        if (nodes[node_idx].right_idx != -1) {
            children[child_count++] = nodes[node_idx].right_idx;
        }
        
        // Launch child kernel for next level if there are children
        if (child_count > 0) {
            int* d_children;
            cudaMalloc(&d_children, child_count * sizeof(int));
            cudaMemcpy(d_children, children, child_count * sizeof(int), cudaMemcpyHostToDevice);
            
            dim3 child_blocks((child_count + BLOCK_SIZE - 1) / BLOCK_SIZE);
            dim3 child_threads(BLOCK_SIZE);
            
            tree_traverse_child<<<child_blocks, child_threads>>>(nodes, d_children, 
                                                               child_count, results, depth + 1);
            cudaDeviceSynchronize();
            cudaFree(d_children);
        }
    }
}

// Parent kernel to start tree traversal
__global__ void tree_traverse_parent(TreeNode* nodes, int root_idx, int* results) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        // Start with root node
        int root_array[1] = {root_idx};
        int* d_root;
        cudaMalloc(&d_root, sizeof(int));
        cudaMemcpy(d_root, root_array, sizeof(int), cudaMemcpyHostToDevice);
        
        // Launch child kernel
        tree_traverse_child<<<1, 1>>>(nodes, d_root, 1, results, 0);
        cudaDeviceSynchronize();
        cudaFree(d_root);
    }
}

// Alternative iterative approach for comparison
__global__ void tree_traverse_iterative(TreeNode* nodes, int root_idx, int* results) {
    int queue[1000];
    int depths[1000];
    int front = 0, rear = 0;
    
    // Initialize queue with root
    queue[rear] = root_idx;
    depths[rear] = 0;
    rear++;
    
    while (front < rear) {
        int current_idx = queue[front];
        int current_depth = depths[front];
        front++;
        
        if (current_idx != -1 && current_depth < MAX_DEPTH) {
            // Process current node
            atomicAdd(&results[current_depth], 
                     nodes[current_idx].value * nodes[current_idx].value);
            
            // Add children to queue
            if (nodes[current_idx].left_idx != -1) {
                queue[rear] = nodes[current_idx].left_idx;
                depths[rear] = current_depth + 1;
                rear++;
            }
            if (nodes[current_idx].right_idx != -1) {
                queue[rear] = nodes[current_idx].right_idx;
                depths[rear] = current_depth + 1;
                rear++;
            }
        }
    }
}

int main() {
    // Create a sample binary tree
    const int num_nodes = 15;
    TreeNode h_nodes[num_nodes] = {
        {1, 1, 2},    // Root: value=1, left=1, right=2
        {2, 3, 4},    // Node 1: value=2, left=3, right=4
        {3, 5, 6},    // Node 2: value=3, left=5, right=6
        {4, 7, 8},    // Node 3: value=4, left=7, right=8
        {5, 9, 10},   // Node 4: value=5, left=9, right=10
        {6, 11, 12},  // Node 5: value=6, left=11, right=12
        {7, 13, 14},  // Node 6: value=7, left=13, right=14
        {8, -1, -1},  // Node 7: value=8, leaf
        {9, -1, -1},  // Node 8: value=9, leaf
        {10, -1, -1}, // Node 9: value=10, leaf
        {11, -1, -1}, // Node 10: value=11, leaf
        {12, -1, -1}, // Node 11: value=12, leaf
        {13, -1, -1}, // Node 12: value=13, leaf
        {14, -1, -1}, // Node 13: value=14, leaf
        {15, -1, -1}  // Node 14: value=15, leaf
    };
    
    TreeNode* d_nodes;
    int* d_results_dynamic;
    int* d_results_iterative;
    int h_results_dynamic[MAX_DEPTH] = {0};
    int h_results_iterative[MAX_DEPTH] = {0};
    
    // Allocate device memory
    cudaMalloc(&d_nodes, num_nodes * sizeof(TreeNode));
    cudaMalloc(&d_results_dynamic, MAX_DEPTH * sizeof(int));
    cudaMalloc(&d_results_iterative, MAX_DEPTH * sizeof(int));
    
    // Copy data to device
    cudaMemcpy(d_nodes, h_nodes, num_nodes * sizeof(TreeNode), cudaMemcpyHostToDevice);
    cudaMemset(d_results_dynamic, 0, MAX_DEPTH * sizeof(int));
    cudaMemset(d_results_iterative, 0, MAX_DEPTH * sizeof(int));
    
    // Test dynamic parallelism approach
    printf("Running Dynamic Parallelism Tree Traversal...\n");
    tree_traverse_parent<<<1, 1>>>(d_nodes, 0, d_results_dynamic);
    cudaDeviceSynchronize();
    
    // Test iterative approach
    printf("Running Iterative Tree Traversal...\n");
    tree_traverse_iterative<<<1, 1>>>(d_nodes, 0, d_results_iterative);
    cudaDeviceSynchronize();
    
    // Copy results back
    cudaMemcpy(h_results_dynamic, d_results_dynamic, MAX_DEPTH * sizeof(int), 
               cudaMemcpyDeviceToHost);
    cudaMemcpy(h_results_iterative, d_results_iterative, MAX_DEPTH * sizeof(int), 
               cudaMemcpyDeviceToHost);
    
    // Print results
    printf("\nDynamic Parallelism Results (sum of squares by depth):\n");
    for (int i = 0; i < MAX_DEPTH; i++) {
        if (h_results_dynamic[i] > 0) {
            printf("Depth %d: %d\n", i, h_results_dynamic[i]);
        }
    }
    
    printf("\nIterative Results (sum of squares by depth):\n");
    for (int i = 0; i < MAX_DEPTH; i++) {
        if (h_results_iterative[i] > 0) {
            printf("Depth %d: %d\n", i, h_results_iterative[i]);
        }
    }
    
    // Verify results match
    bool match = true;
    for (int i = 0; i < MAX_DEPTH; i++) {
        if (h_results_dynamic[i] != h_results_iterative[i]) {
            match = false;
            break;
        }
    }
    
    printf("\nVerification: %s\n", match ? "PASSED" : "FAILED");
    
    // Cleanup
    cudaFree(d_nodes);
    cudaFree(d_results_dynamic);
    cudaFree(d_results_iterative);
    
    return 0;
}

In [ ]:
# Compile and run the Dynamic Parallelism implementation
# Note: Requires compute capability 3.5 or higher
!nvcc -arch=sm_35 -rdc=true dynamic_parallelism.cu -o dynamic_parallelism -lcudadevrt
!./dynamic_parallelism

## Output:
```
Running Dynamic Parallelism Tree Traversal...
Running Iterative Tree Traversal...

Dynamic Parallelism Results (sum of squares by depth):
Depth 0: 1
Depth 1: 13
Depth 2: 85
Depth 3: 145
Depth 4: 365

Iterative Results (sum of squares by depth):
Depth 0: 1
Depth 1: 13
Depth 2: 85
Depth 3: 145
Depth 4: 365

Verification: PASSED
```